In [1]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.types import StringType, DoubleType
import time

spark = SparkSession.builder.appName("UDF_Lab").getOrCreate()

# Генерируем 1 млн точек данных
df = spark.range(0, 1000000).select(
    (F.rand() * 100).alias("temperature")  # температрура от 0 до 100
)

print(f"Rows: {df.count()}")

Rows: 1000000


### Standard Python UDF (Медленный способ)

In [3]:
# 1. Пишем чистую функцию Python
def categorize_temp(temp):
    if temp < 10:
        return "Cold"
    elif temp < 25:
        return "Comfort"
    else:
        return "Hot"

# 2. Регистрируем как UDF (обязательно указываем тип возврата!)
categorize_udf = F.udf(categorize_temp, StringType())

# Применяем
start = time.time()
df.withColumn(
    "category",
    categorize_udf(F.col("temperature"))).write.format("noop").mode("overwrite").save()

print(f"Standard UDF Time: {time.time() - start:.2f} seconds")

Standard UDF Time: 1.80 seconds


### Pandas UDF (Быстрый способ)

In [4]:
from pyspark.sql.functions import pandas_udf
import pandas as pd

# Декоратор региструет функцию. Тип указываем сразу.
# В новейших версиях Spark (3.0+) синтаксис hint типов Python предпочтителен, но строковый тип тоже работает.
@pandas_udf(StringType())
def categorize_pandas_udf(temp_series: pd.Series) -> pd.Series:
    # Мы работаем с вектором (Series), а не с числом!
    return temp_series.apply(
        lambda x: "Cold" if x < 10 else "Comfort" if x < 25 else "Hot"
    )

start = time.time()
df.withColumn("category", categorize_pandas_udf(F.col("temperature"))).write.format("noop").mode("overwrite").save()
print(f"Pandas UDF Time: {time.time() - start:.2f} seconds")

# Ожидание: Pandas UDF должен быть быстрее (иногда в разы, на сложных вычислениях).
# На простых if-else разница может быть небольшой из-за накладных расходов Arrow, но на математике разрыв колоссальный.

Pandas UDF Time: 1.23 seconds


### Нативный способ (Best Practice)

Самый быстрый UDF — это отсутствие UDF. Всегда проверяйте, можно ли сделать это через F.when

In [5]:
start = time.time()
df.withColumn("category", 
    F.when(F.col("temperature") < 10, "Cold")
     .when(F.col("temperature") < 25, "Comfort")
     .otherwise("Hot")
).write.format("noop").mode("overwrite").save()
print(f"Native Spark Time: {time.time() - start:.2f} seconds")

# Это будет быстрее всех.

Native Spark Time: 0.25 seconds
